In [1]:
%matplotlib inline
import torch
import torch.nn.functional as F
import os, sys
from pathlib import Path
from TCN.mnist_pixel.utils import data_generator
from TCN.mnist_pixel.model import TCN
import numpy as np
import matplotlib.pyplot as plt
import pickle
from IPython import display
import ipywidgets as widgets
from PIL import Image
import io
import cv2
import time

In [2]:
model_path = Path('./TCN/mnist_pixel/models')
data_path = Path('./TCN/mnist_pixel/data/mnist')
model_name = 'aug_k7l6'
batch_size = 1
in_channels = 1
n_classes = 10

args = pickle.load(open(model_path / (model_name+'_args.pkl'), 'rb'))
print(args)
channel_sizes = [args.nhid] * args.levels

print(model_path / (model_name+'.pt'))

_, test_loader = data_generator(data_path, batch_size)
model = TCN(in_channels, n_classes, channel_sizes, kernel_size=args.ksize, trt=True)
model.load_state_dict(torch.load(model_path / (model_name+'.pt')), strict=False)
model.eval()

print(model.receptive_field)

if torch.cuda.is_available():
    model.cuda()
    

Namespace(batch_size=64, clip=-1, cuda=True, dropout=0.05, epochs=50, ksize=7, levels=6, log_interval=100, lr=0.002, modelname='aug_k7l6', nhid=25, optim='Adam', permute=False, savedir=PosixPath('models'), savemodel=True, seed=-1, seq_augment=True)
TCN/mnist_pixel/models/aug_k7l6.pt


/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


757


In [3]:
test_t = torch.zeros(1,1,784).cuda()
test_out = model(test_t)

In [4]:
num_samples = 0
# fig, ax = plt.subplots(1,1)

im_queue = [0 for i in range(28*28)]
data_queue = torch.zeros((1,1,28*28)).cuda()
curr_im = np.array(im_queue, dtype=np.uint8).reshape((28,28))
_, encoded_image = cv2.imencode('.png', curr_im)
im_bytes = encoded_image.tobytes()
im_disp = widgets.Image(value=im_bytes, width=200, height=200)

true_val = widgets.Label(value=f'True Label: N/A')
pred_val = widgets.Label(value=f'Predicted Label: N/A')
label_disp = widgets.VBox((true_val, pred_val))

display.display(im_disp)
display.display(label_disp)

for data, target in test_loader:
    im = data.squeeze().cpu().detach().numpy()
    rows, cols = im.shape
    im = (im - im.min())
    im = (im/im.max() * 255).astype('uint8')
    curr_im = np.ones(im.shape, dtype=np.uint8)*255
    
    true_val.value = f'True Label: {target.item()}'

    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    
    data = data.view(data.size()[0], 1, -1)
    
    for i in range(data.size()[2]):
        num_samples += 1
        curr_row = i // cols
        curr_col = i % cols
        
        im_queue.append(im[curr_row,curr_col])
        im_queue = im_queue[1:]
        
        data_queue = data_queue.roll(-1, 2)
        data_queue[:,:,-1] = data[:,:,i]
        
        curr_im = np.array(im_queue, dtype=np.uint8).reshape((28,28))
        _, encoded_image = cv2.imencode('.png', curr_im)
        im_bytes = encoded_image.tobytes()
        im_disp.value = im_bytes
        
        # output = model(data[:,:,i].view(data.size()[0], data.size()[1], 1))
        output = model(data_queue)
        pred = output.max(1, keepdim=True)[1] #max returns values and indices
        
        if num_samples > model.receptive_field:
            pred_val.value = f'Predicted Label: {pred.item()}'
    
        time.sleep(0.001)
    
    
    for i in np.zeros(np.random.randint(50, 200)):
        im_queue.append(0)
        im_queue = im_queue[1:]
        data_queue = data_queue.roll(-1, 2)
        data_queue[:,:,-1] = i
        curr_im = np.array(im_queue, dtype=np.uint8).reshape((28,28)) 
        _, encoded_image = cv2.imencode('.png', curr_im)
        im_bytes = encoded_image.tobytes()
        im_disp.value = im_bytes
        true_val.value = f'True Label: N/A'
        num_samples += 1
        # output = model(torch.tensor([i], dtype=torch.float).cuda().view(1, 1, 1))
        output = model(data_queue)
        pred = output.max(1, keepdim=True)[1] #max returns values and indices
        if num_samples > model.receptive_field:
            pred_val.value = f'Predicted Label: {pred.item()}'
        


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x1c\x00\x00\x00\x1c\x08\x00\x00\x00\x00Wf\x80H\x…

KeyboardInterrupt: 

In [2]:
import onnx
import onnxruntime as ort
from onnx import helper, shape_inference
print(ort.__version__)

1.7.0


In [6]:
onnx_model = torch.onnx.export(
    model,
    test_t,
    f'{model_path}/{model_name}.onnx',
    opset_version=12,
    do_constant_folding=True,
    export_params=True,
    input_names = ['input'],
    output_names = ['output'],
    verbose=True
)


graph(%input : Float(1, 1, 784, strides=[784, 784, 1], requires_grad=0, device=cuda:0),
      %tcn.network.0.conv1.bias : Float(25, strides=[1], requires_grad=1, device=cuda:0),
      %tcn.network.0.conv2.bias : Float(25, strides=[1], requires_grad=1, device=cuda:0),
      %tcn.network.0.downsample.weight : Float(25, 1, 1, strides=[1, 1, 1], requires_grad=1, device=cuda:0),
      %tcn.network.0.downsample.bias : Float(25, strides=[1], requires_grad=1, device=cuda:0),
      %tcn.network.1.conv1.bias : Float(25, strides=[1], requires_grad=1, device=cuda:0),
      %tcn.network.1.conv2.bias : Float(25, strides=[1], requires_grad=1, device=cuda:0),
      %tcn.network.2.conv1.bias : Float(25, strides=[1], requires_grad=1, device=cuda:0),
      %tcn.network.2.conv2.bias : Float(25, strides=[1], requires_grad=1, device=cuda:0),
      %tcn.network.3.conv1.bias : Float(25, strides=[1], requires_grad=1, device=cuda:0),
      %tcn.network.3.conv2.bias : Float(25, strides=[1], requires_grad=1, devi

In [7]:
onnx_model = onnx.load(model_path / (model_name+'.onnx'))
onnx.checker.check_model(onnx_model)
inferred_model = shape_inference.infer_shapes(onnx_model)
onnx.checker.check_model(inferred_model)

In [8]:
onnx_path = model_path / (model_name+'.onnx')
ort_session = ort.InferenceSession(str(onnx_path))

In [9]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(test_t)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(test_out), ort_outs[0], rtol=1e-03, atol=1e-05)
print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!


In [10]:
num_samples = 0
# fig, ax = plt.subplots(1,1)

im_queue = [0 for i in range(28*28)]
data_queue = torch.zeros((1,1,28*28)).cuda()
curr_im = np.array(im_queue, dtype=np.uint8).reshape((28,28))
_, encoded_image = cv2.imencode('.png', curr_im)
im_bytes = encoded_image.tobytes()
im_disp = widgets.Image(value=im_bytes, width=200, height=200)

true_val = widgets.Label(value=f'True Label: N/A')
pred_val = widgets.Label(value=f'Predicted Label: N/A')
label_disp = widgets.VBox((true_val, pred_val))

display.display(im_disp)
display.display(label_disp)

for data, target in test_loader:
    im = data.squeeze().cpu().detach().numpy()
    rows, cols = im.shape
    im = (im - im.min())
    im = (im/im.max() * 255).astype('uint8')
    curr_im = np.ones(im.shape, dtype=np.uint8)*255
    
    true_val.value = f'True Label: {target.item()}'

    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    
    data = data.view(data.size()[0], 1, -1)
    
    for i in range(data.size()[2]):
        num_samples += 1
        curr_row = i // cols
        curr_col = i % cols
        
        im_queue.append(im[curr_row,curr_col])
        im_queue = im_queue[1:]
        
        data_queue = data_queue.roll(-1, 2)
        data_queue[:,:,-1] = data[:,:,i]
        
        curr_im = np.array(im_queue, dtype=np.uint8).reshape((28,28))
        _, encoded_image = cv2.imencode('.png', curr_im)
        im_bytes = encoded_image.tobytes()
        im_disp.value = im_bytes
        
        ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(data_queue)}
        ort_outs = ort_session.run(None, ort_inputs)
        pred = ort_outs[0].argmax(axis=1)[0]
        # output = model(data_queue)
        # pred = output.max(1, keepdim=True)[1] #max returns values and indices
        
        if num_samples > model.receptive_field:
            pred_val.value = f'Predicted Label: {pred.item()}'
    
        time.sleep(0.001)
    
    
    for i in np.zeros(np.random.randint(50, 200)):
        im_queue.append(0)
        im_queue = im_queue[1:]
        data_queue = data_queue.roll(-1, 2)
        data_queue[:,:,-1] = i
        
        curr_im = np.array(im_queue, dtype=np.uint8).reshape((28,28)) 
        _, encoded_image = cv2.imencode('.png', curr_im)
        im_bytes = encoded_image.tobytes()
        im_disp.value = im_bytes
        true_val.value = f'True Label: N/A'
        num_samples += 1

        ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(data_queue)}
        ort_outs = ort_session.run(None, ort_inputs)
        pred = ort_outs[0].argmax(axis=1)[0]
        
        if num_samples > model.receptive_field:
            pred_val.value = f'Predicted Label: {pred.item()}'
        

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x1c\x00\x00\x00\x1c\x08\x00\x00\x00\x00Wf\x80H\x…

KeyboardInterrupt: 

In [11]:
import tensorrt as trt

In [13]:
onnx_model_path = f'{model_path}/{model_name}.onnx'
EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
with trt.Builder(TRT_LOGGER) as builder, builder.create_network(EXPLICIT_BATCH) as network, trt.OnnxParser(network, TRT_LOGGER) as parser:
    with open(onnx_model_path, 'rb') as model:
        if not parser.parse(model.read()):
            for error in range(parser.num_errors):
                print(parser.get_error(error))

NameError: name 'TRT_LOGGER' is not defined